In [ ]:
import pandas as pd
from scipy.optimize import minimize
import numpy as np

In [ ]:
idx_start, idx_end = 60, 125#60, 125 #0, 60
# for fixing 2016-2020, use 0 and 60
# for fixing 2021-2025, use 61 and 125

In [ ]:
wgts = pd.read_excel("China CPI Solver.xlsx", engine="openpyxl", sheet_name="Weight", index_col=0).iloc[idx_start:idx_end]
yoy = pd.read_excel("China CPI Solver.xlsx", engine="openpyxl", sheet_name="YoY", index_col=0).iloc[idx_start:idx_end]
mom = pd.read_excel("China CPI Solver.xlsx", engine="openpyxl", sheet_name="MoM", index_col=0).iloc[idx_start:idx_end]

print("Start and End dates: %.10s and %.10s"%(wgts.index[0], wgts.index[-1]))

In [ ]:
sectors = wgts.columns[1:].tolist()
actual_y = yoy["Total"].values
actual_m = mom["Total"].values
init_guess = wgts.iloc[0,1:].to_numpy().tolist()

In [ ]:
sectors

In [ ]:
def opt_func(w):
    w_tmp = w
    chg_y = yoy.iloc[:,1:].values
    chg_m = mom.iloc[:,1:].values
    wgt_tmp_y = pd.DataFrame([w_tmp]*len(chg_y)).values
    wgt_tmp_m = pd.DataFrame([w_tmp]*len(chg_m)).values
    
    s_y = (wgt_tmp_y*chg_y/100).sum(axis=1)-actual_y
    s_m = (wgt_tmp_m*chg_m/100).sum(axis=1)-actual_m
    
    return np.sqrt(np.sum(s_y**2))+np.sqrt(np.sum(s_m**2))

In [ ]:
cons = (
    {"type":"ineq", "fun":lambda x:abs(sum(x)-100)}
)
bnds = tuple([(0,100) for i in range(len(sectors))])

In [ ]:
res = minimize(opt_func, x0=init_guess, method="SLSQP", bounds=bnds,  constraints=cons, tol=1e-9, options={"disp":True})

In [ ]:
pd.DataFrame([res.x],columns=sectors).to_csv("results_weight.csv")

In [ ]:
res.x